In [103]:
from itertools import product
import pandas as pd
import time
from IPython.display import display, clear_output
from plotnine import *

df = pd.read_csv('data/cdcpniweekly/combined.csv')
df = df[['Year', 'Week', 'Pneumonia Deaths', 'Influenza Deaths', 'snapshot']]
df.columns = ['year', 'week', 'deaths_pn', 'deaths_in', 'snap']
df['snap'] = df['snap'].astype(str)
df = df[df['year']>2010]
df.head()

,year,week,deaths_pn,deaths_in,snap
12,2011,1,4817,53,201601
13,2011,2,4725,66,201601
14,2011,3,4715,67,201601
15,2011,4,4820,88,201601
16,2011,5,4705,106,201601


In [202]:
# expand grid: https://stackoverflow.com/questions/12130883/r-expand-grid-function-in-python
# since data is missing, create a data frame of all combos of real year/week + snapshot
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], 
                        columns=dictionary.keys())
dictionary = {'year': df.year.unique(),
              'week': df.week.unique(),
              'snap': df.snap.unique(),}
df_all = expand_grid(dictionary)

# merge with data so we end up with NA where we have gaps
# use fillna() to fill gaps... seems good in theory but still has glitches
df_all = df_all.merge(df, on=['year', 'week', 'snap'], how='outer')
df_all['time'] = df_all['year'].astype(str) + df_all['week'].astype(str).str.pad(width=2, fillchar='0')
df_all = df_all[(df_all['snap'] >= df_all['time'])]
df_all['deaths_pn'] = df_all['deaths_pn'].fillna(method='bfill')
df_all = df_all.sort_values(['snap', 'year', 'week'])
df_all = df_all[df_all['week'] < 53]
df_all

,year,week,snap,deaths_pn,deaths_in,time
0,2011,1,201601,4817.0,53.0,201101
218,2011,2,201601,4725.0,66.0,201102
436,2011,3,201601,4715.0,67.0,201103
654,2011,4,201601,4820.0,88.0,201104
872,2011,5,201601,4705.0,106.0,201105
...,...,...,...,...,...,...
105947,2020,9,202013,3390.0,553.0,202009
106165,2020,10,202013,3404.0,529.0,202010
106383,2020,11,202013,3203.0,491.0,202011
106601,2020,12,202013,2930.0,331.0,202012


In [205]:
### for testing, you can subset to only the first n snaps
#for i, s in enumerate(df_all.snap.unique()[0:100]):
for i, s in enumerate(df_all.snap.unique()):
    df_sub = df_all[df_all['snap'] == s]
    p = ggplot(df_sub, aes(x='week', y='deaths_pn', color='factor(year)', group='year')) + geom_line()
    p = p + scale_color_discrete(name='year',
                                 limits=[2011+i for i in range(10)])
    p = p + scale_x_continuous(name=' ', breaks=[1, 14, 35, 48],
                               labels=['Jan', 'Apr', 'Sep', 'Dec'])
    p = p + scale_y_continuous(name='reported deaths fron pneumonia', limits=[0, 6500])
    p = p + theme_bw() + theme(panel_grid_major_x=element_blank(), plot_title=element_text(ha='right'))
    p = p + ggtitle(f'data as of: {s[0:4]}, week {s[-2:]}')
    
    ### uncomment these for viewing
    #clear_output(wait=True)
    #display(p)
    #time.sleep(0.01)
    
    ### uncomment to save out plots; might have to make this directory
    p.save(f'./plots/{i:03}.png', dpi=150, width=9, height=6, verbose=False)

/home/jwhendy/.local/lib/python3.8/site-packages/plotnine/geoms/geom_path.py:75: PlotnineWarning: geom_path: Removed 1 rows containing missing values.
  warn(msg.format(n1-n2), PlotnineWarning)


In [198]:
cli = ['convert -loop -1']
cli.extend(f'-delay 0.1 {f}' for f in os.listdir('./plots') if f != 'pnu_deaths.gif' )
cli.pop(-1)
cli.append('-delay 300 217.png ../pnu_deaths.gif')

### copy/paste into a terminal to use with convert from imagemagic
' '.join(cli)

'convert -loop -1 -delay 0.1 000.png -delay 0.1 001.png -delay 0.1 002.png -delay 0.1 003.png -delay 0.1 004.png -delay 0.1 005.png -delay 0.1 006.png -delay 0.1 007.png -delay 0.1 008.png -delay 0.1 009.png -delay 0.1 010.png -delay 0.1 011.png -delay 0.1 012.png -delay 0.1 013.png -delay 0.1 014.png -delay 0.1 015.png -delay 0.1 016.png -delay 0.1 017.png -delay 0.1 018.png -delay 0.1 019.png -delay 0.1 020.png -delay 0.1 021.png -delay 0.1 022.png -delay 0.1 023.png -delay 0.1 024.png -delay 0.1 025.png -delay 0.1 026.png -delay 0.1 027.png -delay 0.1 028.png -delay 0.1 029.png -delay 0.1 030.png -delay 0.1 031.png -delay 0.1 032.png -delay 0.1 033.png -delay 0.1 034.png -delay 0.1 035.png -delay 0.1 036.png -delay 0.1 037.png -delay 0.1 038.png -delay 0.1 039.png -delay 0.1 040.png -delay 0.1 041.png -delay 0.1 042.png -delay 0.1 043.png -delay 0.1 044.png -delay 0.1 045.png -delay 0.1 046.png -delay 0.1 047.png -delay 0.1 048.png -delay 0.1 049.png -delay 0.1 050.png -delay 0.1 05